In [55]:
# 导入进行线性回归的包
import pandas as pd

test_df = pd.read_csv("/Users/datou/PycharmProjects/model/JianyangSta/JY_ensemble/data/建阳站多模型验证期结果.csv", parse_dates=["Date"])
test_df_copy = test_df
test_df_copy = test_df_copy.iloc[:, :2]
test_df.reset_index(drop=True)
test_df = test_df.set_index('Date')
test_df_copy

,Date,Qobs
0,2013-03-27,357.285417
1,2013-03-28,266.375000
2,2013-03-29,292.975000
3,2013-03-30,279.295833
4,2013-03-31,263.750000
...,...,...
2039,2018-10-26,39.116250
2040,2018-10-27,46.711250
2041,2018-10-28,70.970417
2042,2018-10-29,64.495417


In [56]:
"""
读取训练数据
"""
train_df = pd.read_csv("/Users/datou/PycharmProjects/model/JianyangSta/JY_ensemble/data/建阳站多模型率定期结果.csv",parse_dates=["Date"])
train_df_copy = train_df
train_df_copy = train_df_copy.iloc[:, :2]
train_df.reset_index(drop=True)
train_df = train_df.set_index('Date')
train_df.head()

,Qobs,XAJ,TVGM,TANK,GR4J,BP,SVR,LSTM,RNN
Date,,,,,,,,,
2000-03-19,146.646083,113.312715,52.709690,180.82450,161.918257,225.542370,151.506776,210.58240,203.54922
2000-03-20,126.275542,77.366574,45.722771,132.35930,151.888716,122.160250,149.861280,140.79071,100.39914
2000-03-21,116.957458,67.217913,43.718240,109.94270,120.206044,92.493935,132.048647,131.23845,104.74185
2000-03-22,107.373583,69.118466,43.801287,97.71193,107.209378,101.336530,131.638650,116.44574,101.35420
2000-03-23,157.025792,112.531645,62.675173,156.76230,128.057335,209.905500,122.400853,198.44711,141.74525


In [57]:
# 缺失值检验
test_df[test_df.isnull() == True].count()
train_df[test_df.isnull() == True].count()

Qobs    0
XAJ     0
TVGM    0
TANK    0
GR4J    0
BP      0
SVR     0
LSTM    0
RNN     0
dtype: int64

In [58]:
# 计算各特征值相关系数
test_df.corr()

,Qobs,XAJ,TVGM,TANK,GR4J,BP,SVR,LSTM,RNN
Qobs,1.000000,0.875529,0.857664,0.860042,0.849503,0.872199,0.805346,0.871112,0.868201
XAJ,0.875529,1.000000,0.944774,0.973836,0.940090,0.961309,0.871350,0.963692,0.962783
TVGM,0.857664,0.944774,1.000000,0.925511,0.921419,0.930495,0.870571,0.925833,0.920119
TANK,0.860042,0.973836,0.925511,1.000000,0.945013,0.964770,0.871530,0.956504,0.965441
GR4J,0.849503,0.940090,0.921419,0.945013,1.000000,0.937331,0.876579,0.943849,0.942643
BP,0.872199,0.961309,0.930495,0.964770,0.937331,1.000000,0.878034,0.985629,0.984861
SVR,0.805346,0.871350,0.870571,0.871530,0.876579,0.878034,1.000000,0.879261,0.881262
LSTM,0.871112,0.963692,0.925833,0.956504,0.943849,0.985629,0.879261,1.000000,0.987849
RNN,0.868201,0.962783,0.920119,0.965441,0.942643,0.984861,0.881262,0.987849,1.000000


In [59]:
train_df.corr()

,Qobs,XAJ,TVGM,TANK,GR4J,BP,SVR,LSTM,RNN
Qobs,1.000000,0.886951,0.845739,0.860932,0.874877,0.908281,0.821433,0.896979,0.891230
XAJ,0.886951,1.000000,0.943284,0.966673,0.949170,0.956846,0.879456,0.961657,0.957769
TVGM,0.845739,0.943284,1.000000,0.916126,0.924165,0.915209,0.897365,0.911430,0.909944
TANK,0.860932,0.966673,0.916126,1.000000,0.941997,0.949169,0.877882,0.951032,0.966454
GR4J,0.874877,0.949170,0.924165,0.941997,1.000000,0.945645,0.894168,0.952722,0.950102
BP,0.908281,0.956846,0.915209,0.949169,0.945645,1.000000,0.869450,0.985705,0.976888
SVR,0.821433,0.879456,0.897365,0.877882,0.894168,0.869450,1.000000,0.881962,0.883635
LSTM,0.896979,0.961657,0.911430,0.951032,0.952722,0.985705,0.881962,1.000000,0.983938
RNN,0.891230,0.957769,0.909944,0.966454,0.950102,0.976888,0.883635,0.983938,1.000000


In [60]:
"""
方案一：两个概念性组合(XAJ+TANK)
"""
from sklearn.linear_model import LinearRegression
from tool import Indicator

# 处理数据
trainX_XAJ = train_df.iloc[:, 1]
trainX_TANK = train_df.iloc[:, 3]
trainY1 = train_df.iloc[:, :1]
trainX1 = pd.merge(trainX_XAJ, trainX_TANK, on="Date", how="left")
testX_XAJ = test_df.iloc[:, 1]
testX_TANK = test_df.iloc[:, 3]
testY1 = test_df.iloc[:, :1]
testX1 = pd.merge(testX_XAJ, testX_TANK, on="Date", how="left")

#线性拟合
model = LinearRegression()
model.fit(trainX1, trainY1)
# 截距
a = model.intercept_
# 回归系数
b = model.coef_
print("最佳拟合线：截距", str(a), "回归系数：", str(b))

#结果输出
train_pred1 = model.predict(trainX1)
train_pred1 = pd.DataFrame(train_pred1)
train_pred1.columns = ["MLR1"]
test_pred1 = model.predict(testX1)
test_pred1 = pd.DataFrame(test_pred1)
test_pred1.columns = ["MLR1"]
# 数据输出

#预报评估
Indicator.eval(train_pred1["MLR1"], train_df_copy["Qobs"])
Indicator.eval(test_pred1["MLR1"], test_df_copy["Qobs"])

最佳拟合线：截距 [1.18494941] 回归系数： [[0.91418182 0.06517077]]
MAE:52.076544434485726
RMSE:120.22688177232556
NSE:0.7868725009216944
RE:1.981398533758699e-14
Qmaxe:0.2010346813084004
Qmine:0.38671344306032124
MAE:58.217794729090464
RMSE:137.16761105409284
NSE:0.765841414378032
RE:-4.830865018223892
Qmaxe:0.24866657637803935
Qmine:0.5966422535028254


(58.217794729090464,
 137.16761105409284,
 0.765841414378032,
 -4.830865018223892,
 0.24866657637803935,
 0.5966422535028254)

In [65]:
"""
方案二：两个黑箱组合(LSTM+BP)
"""
from sklearn.linear_model import LinearRegression
from tool import Indicator

# 处理数据
trainX_BP = train_df.iloc[:, 5]
trainX_LSTM = train_df.iloc[:, 7]
trainY2 = train_df.iloc[:, :1]
trainX2 = pd.merge(trainX_BP, trainX_LSTM, on="Date", how="left")
testX_BP = test_df.iloc[:, 5]
testX_LSTM = test_df.iloc[:, 7]
testY2 = test_df.iloc[:, :1]
testX2 = pd.merge(testX_BP, testX_LSTM, on="Date", how="left")

#线性拟合
model = LinearRegression()
model.fit(trainX2, trainY2)
# 截距
a = model.intercept_
# 回归系数
b = model.coef_
print("最佳拟合线：截距", str(a), "回归系数：", str(b))

#结果输出
train_pred2 = model.predict(trainX2)
train_pred2 = pd.DataFrame(train_pred2)
train_pred2.columns = ["MLR2"]
test_pred2 = model.predict(testX2)
test_pred2 = pd.DataFrame(test_pred2)
test_pred2.columns = ["MLR2"]
# 数据输出

#预报评估
Indicator.eval(train_pred2["MLR2"], train_df_copy["Qobs"])
Indicator.eval(test_pred2["MLR2"], test_df_copy["Qobs"])

最佳拟合线：截距 [-17.77907642] 回归系数： [[1.08539566 0.07179535]]
MAE:44.28875596011188
RMSE:108.92017421905183
NSE:0.8250745592593287
RE:4.739387916980941e-15
Qmaxe:0.003472667297840535
Qmine:1.300165916108148
MAE:54.39888664999484
RMSE:138.6308576512689
NSE:0.7608189564238417
RE:-4.504770594151672
Qmaxe:0.24524022302509815
Qmine:0.223903392128479


(54.39888664999484,
 138.6308576512689,
 0.7608189564238417,
 -4.504770594151672,
 0.24524022302509815,
 0.223903392128479)

In [62]:
"""
方案三：LSTM+BP+TANK+XAJ
"""
from sklearn.linear_model import LinearRegression
from tool import Indicator

# 处理数据
trainY3 = train_df.iloc[:, :1]
trainX3 = pd.merge(trainX1, trainX2, on="Date", how="left")
testY3 = test_df.iloc[:, :1]
testX3 = pd.merge(testX1, testX2, on="Date", how="left")

#线性拟合
model = LinearRegression()
model.fit(trainX3, trainY3)
# 截距
a = model.intercept_
# 回归系数
b = model.coef_
print("最佳拟合线：截距", str(a), "回归系数：", str(b))

#结果输出
train_pred3 = model.predict(trainX3)
train_pred3 = pd.DataFrame(train_pred3)
train_pred3.columns = ["MLR3"]
test_pred3 = model.predict(testX3)
test_pred3 = pd.DataFrame(test_pred3)
test_pred3.columns = ["MLR3"]
# 数据输出

#预报评估
Indicator.eval(train_pred3["MLR3"], train_df_copy["Qobs"])
Indicator.eval(test_pred3["MLR3"], test_df_copy["Qobs"])

最佳拟合线：截距 [-9.32783594] 回归系数： [[ 0.42077727 -0.27536463  1.05137782 -0.08113118]]
MAE:42.45080575770199
RMSE:106.74364305404131
NSE:0.8319957130610461
RE:1.8892722027590692e-14
Qmaxe:0.05547332767591991
Qmine:0.8955168874510557
MAE:51.17562351764073
RMSE:135.3270418189772
NSE:0.772083318299621
RE:-3.957611754227952
Qmaxe:0.24965233144263668
Qmine:0.24936825766402881


(51.17562351764073,
 135.3270418189772,
 0.772083318299621,
 -3.957611754227952,
 0.24965233144263668,
 0.24936825766402881)

In [63]:
test_df_full = pd.concat([test_df_copy, test_pred1, test_pred2, test_pred3], axis=1)
test_df_full = test_df_full.reset_index(drop=True)
test_df_full = test_df_full.set_index('Date')
test_df_full.to_csv("/Users/datou/PycharmProjects/model/JianyangSta/JY_ensemble/data/JY_MLRTest.csv")
train_df_full = pd.concat([train_df_copy, train_pred1, train_pred2, train_pred3], axis=1)
train_df_full = train_df_full.reset_index(drop=True)
train_df_full = train_df_full.set_index('Date')
train_df_full.to_csv("/Users/datou/PycharmProjects/model/JianyangSta/JY_ensemble/data/JY_MLRTrain.csv")
test_df_full

,Qobs,MLR1,MLR2,MLR3
Date,,,,
2013-03-27,357.285417,9.589830,494.457280,419.770756
2013-03-28,266.375000,13.088628,245.629199,212.742119
2013-03-29,292.975000,35.106562,227.685270,204.907933
2013-03-30,279.295833,80.457292,331.036787,287.738013
2013-03-31,263.750000,62.490997,261.493050,220.244884
...,...,...,...,...
2018-10-26,39.116250,61.614000,47.781167,58.809439
2018-10-27,46.711250,86.327924,123.185375,132.833651
2018-10-28,70.970417,82.734492,108.836878,111.672912
